# WebLog Challenge

## Sessionize by IP

### Clean and create temporary folder.  Unzip data.

In [1]:
import shutil
import os
import gzip

scriptpath = os.getcwd()
tempdir = '.tmp'

if os.path.exists(tempdir):
    shutil.rmtree(tempdir)
os.makedirs(tempdir)

extractedlog = 'web.log'
sourcepath = os.path.join(scriptpath, '../data/2015_07_22_mktplace_shop_web_log_sample.log.gz')
destpath = os.path.join(scriptpath, tempdir, extractedlog)
with gzip.open(sourcepath) as src, open(destpath, 'w') as dest:
    content = src.read()
    dest.write(content)

### Build sessions

In [2]:
from analytics.sessionizer import *
from pyspark import SparkContext

sessionizer = Sessionizer(sc)
sessions_rdd = sessionizer.calc_sessions_from_file(destpath, 15)

### Display first 10 sessions

In [3]:
sessions_rdd.take(10)

[{'end': datetime.datetime(2015, 7, 22, 11, 4, 43, 966443, tzinfo=<iso8601.Utc>),
  'id': -3925200981616695027,
  'ip': u'111.93.191.38',
  'requests': [u'paytm.com:443/shop/cart',
   u'paytm.com:443/shop/login',
   u'paytm.com:443/shop/authresponse',
   u'paytm.com:443/shop/v1/frequentorders',
   u'paytm.com:443/shop/cart',
   u'paytm.com:443/shop/cart',
   u'paytm.com:443/shop/cart',
   u'paytm.com:443/shop/cart',
   u'paytm.com:443/shop/user/address',
   u'paytm.com:443/shop/user/address',
   u'paytm.com:443/shop/cart',
   u'paytm.com:443/shop/cart/checkout'],
  'start': datetime.datetime(2015, 7, 22, 11, 3, 6, 822295, tzinfo=<iso8601.Utc>)},
 {'end': datetime.datetime(2015, 7, 22, 6, 58, 36, 823521, tzinfo=<iso8601.Utc>),
  'id': -8762026098901585080,
  'ip': u'49.201.18.231',
  'requests': [u'paytm.com:443/shop/g/electronics/the-electronic-store/ces-offer-on-pen-drives'],
  'start': datetime.datetime(2015, 7, 22, 6, 58, 36, 823521, tzinfo=<iso8601.Utc>)},
 {'end': datetime.datetim

## Average Session Time

In [4]:
avg = sessionizer.average_session_time(sessions_rdd)

print 'Average session time(s):' + str(avg)

Average session time(s):100


## Unique Visits per Sesions

In [8]:
visits_per_session = sessionizer.unique_visits_per_session(sessions_rdd)

it = visits_per_session.iteritems()
for i in range(10):
    session = it.next()
    print 'Session ID:' + str(session[0]) + ', IP:' + session[1]['ip'] + ', Number of Visits:' + str(session[1]['visits']) 

Session ID:-8212373043871481855, IP:122.161.227.47, Number of Visits:1
Session ID:-3066943668411170813, IP:122.177.105.81, Number of Visits:5
Session ID:-2258161021395140602, IP:116.74.48.135, Number of Visits:3
Session ID:6118955677700063233, IP:106.215.154.137, Number of Visits:2
Session ID:4648038607161393161, IP:103.255.145.86, Number of Visits:5
Session ID:-3060370823484801013, IP:59.89.16.137, Number of Visits:3
Session ID:-855732330878205936, IP:43.248.69.104, Number of Visits:3
Session ID:5259766095696363537, IP:59.178.201.68, Number of Visits:2
Session ID:94247279369125891, IP:115.248.179.194, Number of Visits:2
Session ID:1720124910343028756, IP:112.79.36.224, Number of Visits:2


## Active Users

In [11]:
most_active_sessions = sessionizer.find_engaged_users(sessions_rdd)

for session in most_active_sessions[:10]:
    print 'User:' + session['ip'] + ', Duration(s):' + str(session['length']) 

User:52.74.219.71, Duration(s):2069.161909
User:119.81.61.166, Duration(s):2068.84845
User:106.186.23.95, Duration(s):2068.756402
User:125.19.44.66, Duration(s):2068.71247
User:125.20.39.66, Duration(s):2068.320635
User:192.8.190.10, Duration(s):2067.234329
User:54.251.151.39, Duration(s):2067.022968
User:180.211.69.209, Duration(s):2066.960642
User:180.179.213.70, Duration(s):2065.63799
User:203.189.176.14, Duration(s):2065.594344
